# **Adatelőkészítés a pandas csomag segítségével**
Fogarassyné Vathy Ágnes

Az adatelőkészítés fázisa számos különféle műveletet foglal magában. Jelen notebook-ban olyan mintapéldákat tekintünk át, amelyek ezen lehetséges műveletekből igyekeznek minél többet bemutatni, de természetesen nem fedik a teljes témakört.  

## **1. Változók átkódolása**

**Blokk 1.1** A **dummy változók** kialakításának szemléltetéséhez hozzunk létre egy minta dataframe-et, majd alakítsuk át a szín változót dummy változókká.

In [1]:
import pandas as pd

df_minta=pd.DataFrame({'Nev':['Aranka', 'Piroska', 'Józsi', 'Benedek', 'Emese'],
                       'Magasság': ['magas', 'alacsony', 'nagyon magas', 'normál', 'normál'],
                       'Szemszin': ['kék', 'zöld', 'barna', 'zöld', 'kék']})
df_minta

,Nev,Magasság,Szemszin
0,Aranka,magas,kék
1,Piroska,alacsony,zöld
2,Józsi,nagyon magas,barna
3,Benedek,normál,zöld
4,Emese,normál,kék


**Blokk 1.2** Nézzük meg, hogy hogyan működik a **get_dummies** metódus!

In [2]:
pd.get_dummies(df_minta['Szemszin'])

,barna,kék,zöld
0,False,True,False
1,False,False,True
2,True,False,False
3,False,False,True
4,False,True,False


**Blokk 1.3** Alakítsuk át a DataFrame-ben a *Szemszín* oszlopot **one-hot kódolással**!

In [3]:
df_minta = pd.get_dummies(df_minta, columns=['Szemszin'], drop_first=False)
df_minta

,Nev,Magasság,Szemszin_barna,Szemszin_kék,Szemszin_zöld
0,Aranka,magas,False,True,False
1,Piroska,alacsony,False,False,True
2,Józsi,nagyon magas,True,False,False
3,Benedek,normál,False,False,True
4,Emese,normál,False,True,False


**Blokk 1.4** Az előző feladat megoldása referenciakódolással:

In [4]:
df_minta=pd.DataFrame({'Nev':['Aranka', 'Piroska', 'Józsi', 'Benedek', 'Emese'],
                       'Magasság': ['magas', 'alacsony', 'nagyon magas', 'normál', 'normál'],
                       'Szemszin': ['kék', 'zöld', 'barna', 'zöld', 'kék']})

df_minta = pd.get_dummies(df_minta, columns=['Szemszin'], drop_first=True)
df_minta

,Nev,Magasság,Szemszin_kék,Szemszin_zöld
0,Aranka,magas,True,False
1,Piroska,alacsony,False,True
2,Józsi,nagyon magas,False,False
3,Benedek,normál,False,True
4,Emese,normál,True,False


**Blokk 1.5** A one-hot encoding-ot a scikit-learn preprocessing csomagjának **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** osztályával is meg tudjuk valósítani. Bár hosszabb kódolást igényel, az osztály azonban számos egyéb lehetőséget is biztosít, amit érdemes megnézni a tutorial-ban.

Az adatok átalakítása:

In [5]:
from sklearn import preprocessing

df_minta=pd.DataFrame({'Nev':['Aranka', 'Piroska', 'Józsi', 'Benedek', 'Emese'],
                       'Magasság': ['magas', 'alacsony', 'nagyon magas', 'normál', 'normál'],
                       'Szemszin': ['kék', 'zöld', 'barna', 'zöld', 'kék']})

onehot_encoder = preprocessing.OneHotEncoder()

X = onehot_encoder.fit_transform(df_minta['Szemszin'].values.reshape(-1,1)).toarray()

dfOneHot = pd.DataFrame(X, columns = ['Szemszin_'+str(int(i)) for i in range(X.shape[1])])

df_minta = pd.concat([df_minta, dfOneHot], axis=1)
df_minta= df_minta.drop(['Szemszin'], axis=1)

df_minta


,Nev,Magasság,Szemszin_0,Szemszin_1,Szemszin_2
0,Aranka,magas,0.0,1.0,0.0
1,Piroska,alacsony,0.0,0.0,1.0
2,Józsi,nagyon magas,1.0,0.0,0.0
3,Benedek,normál,0.0,0.0,1.0
4,Emese,normál,0.0,1.0,0.0


**Blokk 1.6** A *Szemszin* oszlop átalakítása **[LabelEcoder](https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.LabelEncoder.html)** használatával:

In [6]:
from sklearn.preprocessing import LabelEncoder

df_minta=pd.DataFrame({'Nev':['Aranka', 'Piroska', 'Józsi', 'Benedek', 'Emese'],
                       'Magasság': ['magas', 'alacsony', 'nagyon magas', 'normál', 'normál'],
                       'Szemszin': ['kék', 'zöld', 'barna', 'zöld', 'kék']})

label_encoder = LabelEncoder()
df_minta['Szemszin']= label_encoder.fit_transform(df_minta['Szemszin'])

df_minta


,Nev,Magasság,Szemszin
0,Aranka,magas,1
1,Piroska,alacsony,2
2,Józsi,nagyon magas,0
3,Benedek,normál,2
4,Emese,normál,1


**Blokk 1.7** Az adatok visszaalakítása az illesztett encoder **inverse_transform** metódusával:

In [7]:
df_minta['Szemszin']= label_encoder.inverse_transform(df_minta['Szemszin'])
df_minta

,Nev,Magasság,Szemszin
0,Aranka,magas,kék
1,Piroska,alacsony,zöld
2,Józsi,nagyon magas,barna
3,Benedek,normál,zöld
4,Emese,normál,kék


**Blokk 1.8** A *Magasság* adatok kódolása **ordinális változó**ként:

In [8]:
ordinal_mapper = {'alacsony':1, 'normál':2, 'magas':3, 'nagyon magas':4}

df_minta['Magasság'] = df_minta['Magasság'].replace(ordinal_mapper)
df_minta

/var/folders/p9/yfdwydy169s3mz1hd23bwrtw0000gn/T/ipykernel_43471/1922584183.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_minta['Magasság'] = df_minta['Magasság'].replace(ordinal_mapper)


,Nev,Magasság,Szemszin
0,Aranka,3,kék
1,Piroska,1,zöld
2,Józsi,4,barna
3,Benedek,2,zöld
4,Emese,2,kék


**Blokk 1.9** Az előző feladat megoldása a preprocessing modul **[OridnalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder)** osztályának használatával:

In [9]:
from sklearn.preprocessing import OrdinalEncoder

df_minta=pd.DataFrame({'Nev':['Aranka', 'Piroska', 'Józsi', 'Benedek', 'Emese'],
                       'Magasság': ['magas', 'alacsony', 'nagyon magas', 'normál', 'normál'],
                       'Szemszin': ['kék', 'zöld', 'barna', 'zöld', 'kék']})

# A sorrend a magasság értékek alapján
magassag_sorrend = ['alacsony', 'normál', 'magas', 'nagyon magas']

ordinal_encoder = OrdinalEncoder(categories=[magassag_sorrend])
df_minta['Magasság'] = ordinal_encoder.fit_transform(df_minta[['Magasság']])

df_minta

,Nev,Magasság,Szemszin
0,Aranka,2.0,kék
1,Piroska,0.0,zöld
2,Józsi,3.0,barna
3,Benedek,1.0,zöld
4,Emese,1.0,kék


## **2. Több adatforrás egyesítése**

**Blokk 2.1** Olvassuk be a *szemely1.csv* és a *szemely2.csv* fájlokat, majd egyesítsük őket!

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

In [11]:
import numpy as np
import pandas as pd

szem1 = pd.read_csv('szemely1.csv',
                    sep=';', header=0)
szem2 = pd.read_csv('szemely2.csv',
                    sep=';', header=0)

**Blokk 2.2** Kukkantsunk bele, hogy milyen adatokat tartalmaznak.

In [12]:
szem1.head()

,id,testsuly,testmagassag,eletkor
0,1,82.0,250.0,30
1,2,86.0,169.0,68
2,3,NaN,176.0,63
3,4,NaN,185.0,60
4,5,82.0,168.0,63


In [13]:
szem2.head()

,id,testsuly,testmagassag,eletkor
0,486,NaN,172.0,61
1,487,51.0,157.0,50
2,488,124.0,186.0,94
3,489,86.0,164.0,66
4,490,85.0,179.0,33


**Blokk 2.3** Egyesítsük a 2 állományt!

In [14]:
szem = pd.concat([szem1, szem2], ignore_index=True)
szem

,id,testsuly,testmagassag,eletkor
0,1,82.0,250.0,30
1,2,86.0,169.0,68
2,3,NaN,176.0,63
3,4,NaN,185.0,60
4,5,82.0,168.0,63
...,...,...,...,...
775,776,88.0,182.0,26
776,777,70.0,164.0,50
777,778,51.0,157.0,30
778,779,77.0,165.0,59


**Blokk 2.4** Hány adatot tartalmaznak az eredeti és az egyesített adathalmazok?

In [15]:
print ('Az első állomány: ', szem1.shape)
print ('A második állomány: ', szem2.shape)
print ('Az egyesített állomány: ', szem.shape)

Az első állomány:  (485, 4)
A második állomány:  (295, 4)
Az egyesített állomány:  (780, 4)


**Blokk 2.5** Töröljük a felesleges (szem1, és szem2)*dőlt szöveg* dataframe-eket!

In [16]:
del szem1
del szem2

## **3. Adattisztítás**

### **3.1 Hibás adatok kezelése**

**Blokk 3.1** Vizsgáljuk meg közelebbről a numerikus értékeket tartalmazó oszlopok adatait!  
A numerikus értékek legnagyobb és legkisebb értékeit az **nlargest** és **nsmallest** függvényekkel jeleníthetjük meg.

In [17]:
szem.nlargest(10, 'eletkor')

,id,testsuly,testmagassag,eletkor
187,188,83.0,172.0,199
179,180,82.0,175.0,187
530,531,96.0,180.0,150
700,701,69.0,162.0,150
419,420,81.0,169.0,122
17,18,92.0,161.0,99
26,27,110.0,174.0,99
87,88,108.0,171.0,99
182,183,86.0,155.0,99
352,353,99.0,171.0,99


In [18]:
szem.nsmallest(10, 'eletkor')

,id,testsuly,testmagassag,eletkor
148,149,89.0,173.0,-10
164,165,81.0,161.0,-10
514,515,81.0,169.0,-10
736,737,91.0,172.0,-8
506,507,90.0,175.0,0
511,512,85.0,174.0,0
544,545,65.0,164.0,0
568,569,85.0,181.0,0
628,629,65.0,162.0,0
647,648,68.0,156.0,0


**Blokk 3.2** A nyilvánvaló adathibák esetében egyik lehetséges (és egyben legegyszerűbb) mód a hibás adatok törlése (nem túl sok hibás adat esetén ajánlott csak).   
Töröljük azon életkor adatokat, ahol az életkor nem a [18, 120] intervallumba eseik!

In [19]:
szem.loc[szem.eletkor > 120, 'eletkor'] = np.nan
szem.loc[szem.eletkor < 18, 'eletkor'] = np.nan

Csupán ellenőrzésképpen nézzük meg, hogy pl. a 148-as idexű rekord adatai hogyan módosultak (az előbb láttuk, hogy -10 éves volt).

In [20]:
szem.loc[148]

id              149.0
testsuly         89.0
testmagassag    173.0
eletkor           NaN
Name: 148, dtype: float64

**Blokk 3.3** A *testsuly* adatok esetén legyünk kicsit ügyesebbek! Rendezzük sorba az adatokat, majd nézzük meg hol "ugranak meg" az értékek, s ez alapján töröljük az outlier adatokat tartalmazó rekordokat!

A vizualizációt érdemes plotly-val elkészíteni, így le tudjuk olvasni a grafikonról az értékeket...

In [21]:
import plotly.express as px

szem = szem.sort_values(by='testsuly', ascending=True)
fig = px.line(szem, x='id', y='testsuly')
fig.update_xaxes(type='category') #ez ahhoz kell, hogy az x tengelyt kategórikusnak tekintse, s ne akarja az alapján rendezni az értékeket...
fig.show()

> Leolvasva azt láthatjuk, hogy 50 alatt és 130 felett vannak outlierek. Töröljük így ki a rekordjainkat!

In [22]:
szem.loc[szem.testsuly > 130, 'testsuly'] = np.nan
szem.loc[szem.testsuly < 50, 'testsuly'] = np.nan

**Blokk 3.4** Ismételjük meg az előző eljárást a *testmagassag* oszlopra is!

In [23]:
import plotly.express as px

szem = szem.sort_values(by='testmagassag', ascending=True)
fig = px.line(szem, x='id', y='testmagassag')
fig.update_xaxes(type='category') #ez ahhoz kell, hogy az x tengelyt kategórikusnak tekintse, s ne akarja az alapján rendezni az értékeket...
fig.show()

> A 205-ös és 136-os testmagasságokat még elfogadjuk, ezért töröljük a rekordokat 210-es érték felett és 100-as érték alatt!


In [24]:
szem.loc[szem.testmagassag > 210, 'testmagassag'] = np.nan
szem.loc[szem.testmagassag < 100, 'testmagassag'] = np.nan

### **3.2 Hiányzó adatok kezelése**

#### **Üres adatok lekérdezése**

**Blokk 3.5** Az **isnull** függvény segítségével le tudjuk kérdezni, hogy a DataFrame-ben hol találunk NAN értékeket:

In [25]:
szem.isnull()  #vagy: pd.isnull(szem)

,id,testsuly,testmagassag,eletkor
595,False,False,True,False
124,False,False,True,False
727,False,False,True,False
546,False,False,True,False
284,False,False,True,False
...,...,...,...,...
703,False,False,True,False
469,False,False,True,False
56,False,False,True,False
438,False,False,True,False


**Blokk 3.6** Mivel ez így eléggé átláthatatlan, ezért célszerű továbbfűzni az előző gondolatot és lekérdezni azon sorok indexeit, amelyek tartalmaznak NAN értéket.
Ehhez első lépésben nézzük meg, hogy hogyan lehet lekérdezni, hogy mely sorok/oszlopok tartalmazak NAN értéket. Erre sok lehetőségünk van, megvalósíthatjuk például így is:   
</br>   
Mely oszlopok tartalmaznak NAN adatot?

In [26]:
szem.isnull().any(axis='rows')

id              False
testsuly         True
testmagassag     True
eletkor          True
dtype: bool

És melyik oszlopban hány üres adat van?

In [27]:
szem.isnull().sum()

id               0
testsuly        30
testmagassag    22
eletkor         33
dtype: int64

Mely sorok tartalmaznak NAN adatot?

In [28]:
szem.isnull().any(axis='columns')

595    True
124    True
727    True
546    True
284    True
       ... 
703    True
469    True
56     True
438    True
624    True
Length: 780, dtype: bool

**Blokk 3.7** Az üres adatot tartalmazó sorok indexeit többféleképpen is kilistázhatjuk. Leggyorsabb talán a *szem.isnull().any(axis=1).nonzero()* utasítás, de hasonló eredményt érhetünk el a következőképpen is:

In [29]:
szem[szem.isnull().any(axis=1)].index

Index([595, 124, 727, 546, 284, 462, 600, 251, 477, 719,  72, 422, 647, 333,
       474, 427, 393, 644, 671, 164, 710, 344, 364, 591, 700, 628, 565, 680,
       663, 544,  67, 692,  77, 638, 514, 419, 250,  95, 112, 187, 587, 485,
       368, 736, 305, 148, 765, 511, 744, 561, 585, 179, 506,  86,   2,  51,
       132, 676, 530, 404, 568, 724, 312, 673, 478, 466,   3, 272, 358, 773,
         0, 102,  75, 621, 482,  50, 749, 110, 703, 469,  56, 438, 624],
      dtype='int64')

**Blokk 3.8** Ha arra vagyunk kiváncsiak, hogy hány olyan rekordunk van, amely NaN adatot tartalmaz, akkor azt pl. így kaphatjuk meg:

In [30]:
np.array(szem[szem.isnull().any(axis=1)].index).size

83

#### **Hiányzó adatok pótlása**

A hiányzó adatok pótlását kétféleképpen nézzük meg:
- manuális adatpótlás
- a preprocessing modul által kínált lehetőségek alkalmazásával

**Blokk 3.9** Azért, hogy legyen a későbbiekben majd min dolgozni, készítsünk másolatot a *szem* DataFrame-ről.

In [31]:
szem_copy = szem.copy()

##### **Hiányzó adatok pótlása manuálisan**

**Blokk 3.10** Ha valamely adatot utólagosan beszereztük, akkor könnyen tudjuk  a DataFrame-ben is pótolni.   
Nézzük meg, hogy hol nincs kitöltve a testmagasssg attribútum!


In [32]:
hiany_testmagassag_idx = szem[szem['testmagassag'].isnull()].index.tolist()

print('\n')
print(szem.loc[hiany_testmagassag_idx], '\n')



      id  testsuly  testmagassag  eletkor
595  596      84.0           NaN     45.0
124  125      80.0           NaN     43.0
727  728      78.0           NaN     85.0
546  547      96.0           NaN     72.0
284  285      94.0           NaN     75.0
462  463      81.0           NaN     77.0
600  601      84.0           NaN     27.0
251  252      68.0           NaN     86.0
477  478      55.0           NaN     50.0
0      1      82.0           NaN     30.0
102  103     108.0           NaN     64.0
75    76     100.0           NaN     88.0
621  622     124.0           NaN     86.0
482  483      64.0           NaN     58.0
50    51      66.0           NaN     23.0
749  750      82.0           NaN     26.0
110  111      85.0           NaN     39.0
703  704      88.0           NaN     56.0
469  470     100.0           NaN     47.0
56    57     104.0           NaN     94.0
438  439     115.0           NaN     55.0
624  625       NaN           NaN     63.0 



**Blokk 3.11** Tegyük fel, hogy a *125*-ös *id*-jú személy testmagasságát utólagosan megtudtuk, s ez *180 cm*.

In [33]:
szem.loc[szem['id'] == 125]

,id,testsuly,testmagassag,eletkor
124,125,80.0,NaN,43.0


In [34]:
szem.loc[szem['id'] == 125, 'testmagassag'] = 180
szem.loc[szem['id'] == 125]

,id,testsuly,testmagassag,eletkor
124,125,80.0,180.0,43.0


##### **Feltöltés globális konstanssal**

**Blokk 3.12** Ebben a műveletben a **replace** függvény lesz segítségünkre, a hiányzó értékekre pedig az **np.nan** függvénnyel hivatkozhatunk.  
Cseréljül le a hiányzó *eletkor* adatokat *ismeretlen* értékekre.   
Először vizsgáljuk meg, hogy hol tartalmaz NaN adatokat az *eletkor* mező.

In [35]:
hiany_eletkor_idx = szem[szem['eletkor'].isnull()].index.tolist()

print('\n')
print(szem.loc[hiany_eletkor_idx], '\n')



      id  testsuly  testmagassag  eletkor
72    73       NaN         144.0      NaN
647  648      68.0         156.0      NaN
671  672      61.0         160.0      NaN
164  165      81.0         161.0      NaN
710  711      61.0         161.0      NaN
700  701      69.0         162.0      NaN
628  629      65.0         162.0      NaN
565  566      68.0         163.0      NaN
680  681      61.0         164.0      NaN
663  664      68.0         164.0      NaN
544  545      65.0         164.0      NaN
692  693      67.0         167.0      NaN
514  515      81.0         169.0      NaN
419  420      81.0         169.0      NaN
250  251      81.0         169.0      NaN
95    96      76.0         170.0      NaN
187  188      83.0         172.0      NaN
587  588      77.0         172.0      NaN
368  369      76.0         172.0      NaN
736  737      91.0         172.0      NaN
148  149      89.0         173.0      NaN
765  766      74.0         173.0      NaN
511  512      85.0         174.0

In [36]:
szem.eletkor.replace(np.nan, 'ismeretlen', inplace=True)

print('\n')
print(szem.loc[hiany_eletkor_idx], '\n')



      id  testsuly  testmagassag     eletkor
72    73       NaN         144.0  ismeretlen
647  648      68.0         156.0  ismeretlen
671  672      61.0         160.0  ismeretlen
164  165      81.0         161.0  ismeretlen
710  711      61.0         161.0  ismeretlen
700  701      69.0         162.0  ismeretlen
628  629      65.0         162.0  ismeretlen
565  566      68.0         163.0  ismeretlen
680  681      61.0         164.0  ismeretlen
663  664      68.0         164.0  ismeretlen
544  545      65.0         164.0  ismeretlen
692  693      67.0         167.0  ismeretlen
514  515      81.0         169.0  ismeretlen
419  420      81.0         169.0  ismeretlen
250  251      81.0         169.0  ismeretlen
95    96      76.0         170.0  ismeretlen
187  188      83.0         172.0  ismeretlen
587  588      77.0         172.0  ismeretlen
368  369      76.0         172.0  ismeretlen
736  737      91.0         172.0  ismeretlen
148  149      89.0         173.0  ismeretlen
765  766

/var/folders/p9/yfdwydy169s3mz1hd23bwrtw0000gn/T/ipykernel_43471/2757859190.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





**Blokk 3.13** Mivel a NAN értékeket sokkal könnyebben tudjuk kezelni, ezért írjuk vissza az *eletkor* oszlopba a *NAN* adatokat az *'ismeretlen'* adatok helyett.

In [37]:
szem.eletkor.replace('ismeretlen', np.nan, inplace=True)

print('\n')
print(szem.loc[hiany_eletkor_idx], '\n')



      id  testsuly  testmagassag  eletkor
72    73       NaN         144.0      NaN
647  648      68.0         156.0      NaN
671  672      61.0         160.0      NaN
164  165      81.0         161.0      NaN
710  711      61.0         161.0      NaN
700  701      69.0         162.0      NaN
628  629      65.0         162.0      NaN
565  566      68.0         163.0      NaN
680  681      61.0         164.0      NaN
663  664      68.0         164.0      NaN
544  545      65.0         164.0      NaN
692  693      67.0         167.0      NaN
514  515      81.0         169.0      NaN
419  420      81.0         169.0      NaN
250  251      81.0         169.0      NaN
95    96      76.0         170.0      NaN
187  188      83.0         172.0      NaN
587  588      77.0         172.0      NaN
368  369      76.0         172.0      NaN
736  737      91.0         172.0      NaN
148  149      89.0         173.0      NaN
765  766      74.0         173.0      NaN
511  512      85.0         174.0

/var/folders/p9/yfdwydy169s3mz1hd23bwrtw0000gn/T/ipykernel_43471/2380678280.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### **Hiányzó értékek kezelése a preprocessing modul által nyújtott lehetőségekkel**

**Blokk 3.14** Először töltsük vissza az elmentett DataFrame-ünket, hogy újra legyen adathiányunk...

In [38]:
szem = szem_copy.copy()
print ('Rekordok száma: ', szem.shape[0])

Rekordok száma:  780


**Blokk 3.15** Nézzük meg, hogy a folytonos értékű attribútumok hol nem tartaznak adatot! Írassuk ki az indexüket, majd ezen adatsorokat is!  

In [39]:
hiany_testsuly_idx = szem[szem['testsuly'].isnull()].index.tolist()
hiany_testmagassag_idx = szem[szem['testmagassag'].isnull()].index.tolist()
hiany_eletkor_idx = szem[szem['eletkor'].isnull()].index.tolist()

print ('testsúly hiányzik: ', hiany_testsuly_idx)
print ('testmagasság hiányzik: ', hiany_testmagassag_idx)
print ('életkor hiányzik: ', hiany_eletkor_idx)

#print('\n')
#print(szem.loc[hiany_testsuly_idx], '\n')
#print(szem.loc[hiany_testmagassag_idx], '\n')
#print(szem.loc[hiany_eletkor_idx], '\n')

testsúly hiányzik:  [719, 72, 422, 333, 474, 427, 393, 644, 344, 364, 591, 67, 77, 638, 112, 485, 305, 744, 561, 585, 86, 2, 51, 132, 404, 312, 3, 272, 773, 624]
testmagasság hiányzik:  [595, 124, 727, 546, 284, 462, 600, 251, 477, 0, 102, 75, 621, 482, 50, 749, 110, 703, 469, 56, 438, 624]
életkor hiányzik:  [72, 647, 671, 164, 710, 700, 628, 565, 680, 663, 544, 692, 514, 419, 250, 95, 187, 587, 368, 736, 148, 765, 511, 179, 506, 676, 530, 568, 724, 673, 478, 466, 358]


A Sci-Kit learn **[SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer)** osztálya lehetővé teszi, hogy a hiányzó adatokat kitölthessük.
* Paraméterek:
    * *missing_values* : integer vagy “NaN” -- elhagyható, de default=”NaN”
    * *strategy* : string, optional (default=”mean”)
        Legetséges értékek:
             - "mean": az adott tulajdonság szerinti értékek átlaga
             - "median": az adott tulajdonság szerinti értékek mediánja
             - "most_frequent": az adott tulajdonság szerinti leggyakoribb érték
             
    * *axis* : integer, optional (default=0)
            axis=0, --> oszlopon szerint vizsgál
            axis=1, --> sorok szerint vizsgál

**Blokk 3.16** A hiányzó testmagasságok feltöltése az átlag értékkel:

In [40]:
from sklearn.impute import SimpleImputer

imp_testmag = SimpleImputer(missing_values=np.nan, strategy='median')
szem['testmagassag'] = imp_testmag.fit_transform(szem[['testmagassag']]).ravel()

szem.loc[hiany_testmagassag_idx]

,id,testsuly,testmagassag,eletkor
595,596,84.0,170.0,45.0
124,125,80.0,170.0,43.0
727,728,78.0,170.0,85.0
546,547,96.0,170.0,72.0
284,285,94.0,170.0,75.0
462,463,81.0,170.0,77.0
600,601,84.0,170.0,27.0
251,252,68.0,170.0,86.0
477,478,55.0,170.0,50.0
0,1,82.0,170.0,30.0


Haladóbb lehetőségként érdemes használni az [IterativeImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer)-t, amely a hiányzó értékeket egyéb tulajdonságértékekből becsli. Illetve a [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer)-t, ami a $k$ legközelebbi szomszéd adata alapján határozza meg a hiányzó értékeket. A témához [itt](https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/) találunk egy gyakorlatias útmutatót.

**Blokk 3.17** A hiányzó testsúly értékek feltöltése modell alapján az **IterativeImputer** használatával, az életkor és a testmagasság adatok alapján:

In [41]:
szem.loc[hiany_testsuly_idx]

,id,testsuly,testmagassag,eletkor
719,720,NaN,136.0,22.0
72,73,NaN,144.0,NaN
422,423,NaN,149.0,55.0
333,334,NaN,156.0,92.0
474,475,NaN,156.0,70.0
427,428,NaN,159.0,49.0
393,394,NaN,159.0,18.0
644,645,NaN,160.0,60.0
344,345,NaN,162.0,82.0
364,365,NaN,162.0,40.0


In [42]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Csak a testmagasság és életkor oszlopokat használjuk az imputáláshoz, de a testsúlyt imputáljuk
columns_to_use_for_imputation = ['testsuly', 'testmagassag', 'eletkor']

# IterativeImputer inicializálása
imputer = IterativeImputer()

# Először adjuk át a testsúly, testmagasság és életkor oszlopokat az imputernek
# De csak a testsúly értékeket fogjuk kitölteni
imputed_values = imputer.fit_transform(szem[columns_to_use_for_imputation])

# Csak a testsúly oszlop hiányzó értékeit helyezzük vissza az eredeti dataframe-be
szem['testsuly'] = np.where(szem['testsuly'].isna(), imputed_values[:, 0], szem['testsuly'])

# Hiányzó értékekkel rendelkező sorok kiírása
szem.loc[hiany_testsuly_idx]

,id,testsuly,testmagassag,eletkor
719,720,40.704845,136.0,22.0
72,73,59.697687,144.0,NaN
422,423,64.230995,149.0,55.0
333,334,83.063930,156.0,92.0
474,475,76.011161,156.0,70.0
427,428,72.266741,159.0,49.0
393,394,62.328748,159.0,18.0
644,645,76.789048,160.0,60.0
344,345,85.833663,162.0,82.0
364,365,72.369286,162.0,40.0


#### **Hiányzó értékeket tartalmazó rekordok törlése**

A hiányzó adatok törlése a **dropna** metódussal valósítható meg. Segítségével törölhetünk:  
- sorokat (*axis=0*),
- oszlopokat (*axis=1*)

valamit olyan sorokat/oszlopokat, amelyekben:
- legalább 1 NAN adat van (*how='any'*)
- minden adat NAN (*how='all'*).

**Blokk 3.18** Töröljük azokat a sorokat, melyek csak üres értékeket tartalmaznak! (Bár tudjuk, hogy ilyen most nincs. mert már csak az *eletkor* oszlopban van adathiányunk...)

In [43]:
print ('Törlés előtti rekordok száma: ', szem.shape[0])

szem.dropna(axis=0, how='all', inplace=True)
print ('Törlés utáni rekordok száma: ', szem.shape[0])

Törlés előtti rekordok száma:  780
Törlés utáni rekordok száma:  780


**Blokk 3.19** Töröljük azokat a sorokat, melyek tartalmaznak üres adatot!

In [44]:
print ('Törlés előtti rekordok száma: ', szem.shape[0])

szem.dropna(axis=0, how='any', inplace=True)
print ('Törlés utáni rekordok száma: ', szem.shape[0])

Törlés előtti rekordok száma:  780
Törlés utáni rekordok száma:  747


### **3.3 Duplumok kezelése**

**Blokk 3.20** A duplikált sorokat a **drop_duplicates** metódussal törölhetjük. A metódusban megadhatjuk azt is, hogy mely oszlop egyezőségét vegyük figyelembe. Ha ezt nem tesszük meg, akkor teljes rekordegyezőség esetén történik törlés.

In [45]:
print ('Törlés előtti rekordok száma: ', szem.shape[0])

szem.drop_duplicates(subset=['testsuly', 'testmagassag', 'eletkor'], inplace=True)
print ('Duplikált sorok törlése utáni rekordszám: ', szem.shape[0])

Törlés előtti rekordok száma:  747
Duplikált sorok törlése utáni rekordszám:  727


## **4. Adattranszformáció**

### **4.1 Származtatott oszlop**

**Blokk 4.1** Hozzunk létre egy új oszlopot *bmi* néven, amely az egyes személykre kiszámított testtömeg indexet tartalmazza. A testömeg index a kilogrammban mért testtömeg és a méterben mért tesmagasság négyzetének hányadosa.

In [46]:
szem.head()

,id,testsuly,testmagassag,eletkor
595,596,84.0,170.0,45.0
124,125,80.0,170.0,43.0
727,728,78.0,170.0,85.0
546,547,96.0,170.0,72.0
284,285,94.0,170.0,75.0


In [47]:
szem['bmi'] = szem['testsuly']/((szem['testmagassag']/100)**2)
szem.head(20)

,id,testsuly,testmagassag,eletkor,bmi
595,596,84.000000,170.0,45.0,29.065744
124,125,80.000000,170.0,43.0,27.681661
727,728,78.000000,170.0,85.0,26.989619
546,547,96.000000,170.0,72.0,33.217993
284,285,94.000000,170.0,75.0,32.525952
462,463,81.000000,170.0,77.0,28.027682
600,601,84.000000,170.0,27.0,29.065744
251,252,68.000000,170.0,86.0,23.529412
477,478,55.000000,170.0,50.0,19.031142
719,720,40.704845,136.0,22.0,22.007377


### **4.2 Adatdiszkretizáció**

Az adatok diszretizálásának egyik lehetséges módja a **vödrözési technika** alkalmazása.  
Egyenlő szélességi vödröket a **cut** metódussal, egyenlő mélységű vödröket pedig a **qcut** metódussal  tudunk  kialakítani.
<br>  
**Blokk 4.2** Hozzunk létre *korcsopnev* néven új attribútumot, ahova a személyeket diszkretizáljuk életkoruk alapján *fiatal*, *középkorú* és *idős*  kategóriákba **egyenlő szélességű vödrök** alkalmazásával!

In [48]:
szem['korcsopnev'] = pd.cut(szem.eletkor, 3)
szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,84.000000,170.0,45.0,29.065744,"(17.919, 45.0]"
124,125,80.000000,170.0,43.0,27.681661,"(17.919, 45.0]"
727,728,78.000000,170.0,85.0,26.989619,"(72.0, 99.0]"
546,547,96.000000,170.0,72.0,33.217993,"(45.0, 72.0]"
284,285,94.000000,170.0,75.0,32.525952,"(72.0, 99.0]"
462,463,81.000000,170.0,77.0,28.027682,"(72.0, 99.0]"
600,601,84.000000,170.0,27.0,29.065744,"(17.919, 45.0]"
251,252,68.000000,170.0,86.0,23.529412,"(72.0, 99.0]"
477,478,55.000000,170.0,50.0,19.031142,"(45.0, 72.0]"
719,720,40.704845,136.0,22.0,22.007377,"(17.919, 45.0]"


In [49]:
szem['korcsopnev'] = pd.cut(szem.eletkor, 3,
                            labels=['fiatal', 'kozepkoru', 'idos'])
szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,84.000000,170.0,45.0,29.065744,fiatal
124,125,80.000000,170.0,43.0,27.681661,fiatal
727,728,78.000000,170.0,85.0,26.989619,idos
546,547,96.000000,170.0,72.0,33.217993,kozepkoru
284,285,94.000000,170.0,75.0,32.525952,idos
462,463,81.000000,170.0,77.0,28.027682,idos
600,601,84.000000,170.0,27.0,29.065744,fiatal
251,252,68.000000,170.0,86.0,23.529412,idos
477,478,55.000000,170.0,50.0,19.031142,kozepkoru
719,720,40.704845,136.0,22.0,22.007377,fiatal


**Blokk 4.3** Tegyük ugyanezt meg **egyenlő mélységű vödrök** alkalmazásával, s az eredmény tároljuk el a *korcsopnev2* oszlopban.  

In [50]:
szem['korcsopnev2'] = pd.qcut(szem.eletkor, 3,
                              labels=['fiatal', 'kozepkoru', 'idos'])
szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev,korcsopnev2
595,596,84.000000,170.0,45.0,29.065744,fiatal,kozepkoru
124,125,80.000000,170.0,43.0,27.681661,fiatal,kozepkoru
727,728,78.000000,170.0,85.0,26.989619,idos,idos
546,547,96.000000,170.0,72.0,33.217993,kozepkoru,idos
284,285,94.000000,170.0,75.0,32.525952,idos,idos
462,463,81.000000,170.0,77.0,28.027682,idos,idos
600,601,84.000000,170.0,27.0,29.065744,fiatal,fiatal
251,252,68.000000,170.0,86.0,23.529412,idos,idos
477,478,55.000000,170.0,50.0,19.031142,kozepkoru,kozepkoru
719,720,40.704845,136.0,22.0,22.007377,fiatal,fiatal


Mivel a későbbiekben a *korcsopnev2* oszlopra nincs szükségünk, azért töröljük:

In [51]:
szem.drop('korcsopnev2', axis=1, inplace=True)
szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,84.000000,170.0,45.0,29.065744,fiatal
124,125,80.000000,170.0,43.0,27.681661,fiatal
727,728,78.000000,170.0,85.0,26.989619,idos
546,547,96.000000,170.0,72.0,33.217993,kozepkoru
284,285,94.000000,170.0,75.0,32.525952,idos
462,463,81.000000,170.0,77.0,28.027682,idos
600,601,84.000000,170.0,27.0,29.065744,fiatal
251,252,68.000000,170.0,86.0,23.529412,idos
477,478,55.000000,170.0,50.0,19.031142,kozepkoru
719,720,40.704845,136.0,22.0,22.007377,fiatal


### **4.3. Binarizálás**

Lehetővé teszi, hogy bizonyos adatelőkésztési műveleteknél, olyan határokat huzzunk, amik számunkra megfelelőek, és ennek alkalmazásával tudjuk a rendelkezésre álló adatokat pl. 2 csoportba osztani.

     threshold="X"--> esetében tudjuk megadni azt, hogy mi legyen ez a határ
         - ha az adott érték > X , akkor 1
         - ha az adott érték < = X, akkor 0
         - ha nem állitunk be egy általunk kivánt X értéket küszöbszámnak (threshold), akkor az alapértelmezett 0 érték lép érvénybe


**Blokk 4.4** Egészítsük ki a *szem* DataFrame-et egy *testmag_binarized* oszloppal, amely binarizálva kódolja a testmagasság értékeket 160 cm-es határtékkel!

In [52]:
from sklearn.preprocessing import Binarizer

# Binarizer inicializálása 160 cm-es küszöbértékkel
binarizer = Binarizer(threshold=160)

# A testmagasság oszlop binarizálása
szem['testmagassag_binarizalt'] = binarizer.fit_transform(szem[['testmagassag']])

szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev,testmagassag_binarizalt
595,596,84.000000,170.0,45.0,29.065744,fiatal,1.0
124,125,80.000000,170.0,43.0,27.681661,fiatal,1.0
727,728,78.000000,170.0,85.0,26.989619,idos,1.0
546,547,96.000000,170.0,72.0,33.217993,kozepkoru,1.0
284,285,94.000000,170.0,75.0,32.525952,idos,1.0
462,463,81.000000,170.0,77.0,28.027682,idos,1.0
600,601,84.000000,170.0,27.0,29.065744,fiatal,1.0
251,252,68.000000,170.0,86.0,23.529412,idos,1.0
477,478,55.000000,170.0,50.0,19.031142,kozepkoru,1.0
719,720,40.704845,136.0,22.0,22.007377,fiatal,0.0


**Blokk 4.5** Töröljük a *testmag_binarized* oszlopot!

In [53]:
szem = szem.drop(columns=['testmagassag_binarizalt'])
szem

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,84.000000,170.0,45.0,29.065744,fiatal
124,125,80.000000,170.0,43.0,27.681661,fiatal
727,728,78.000000,170.0,85.0,26.989619,idos
546,547,96.000000,170.0,72.0,33.217993,kozepkoru
284,285,94.000000,170.0,75.0,32.525952,idos
...,...,...,...,...,...,...
703,704,88.000000,170.0,56.0,30.449827,kozepkoru
469,470,100.000000,170.0,47.0,34.602076,kozepkoru
56,57,104.000000,170.0,94.0,35.986159,idos
438,439,115.000000,170.0,55.0,39.792388,kozepkoru


## **5. Tulajdonságok skálázása**

A tulajdonságok skálázását szintén két módon végezzük majd el:
- manuálisan, és
- a preprocessing modul által nyújtott lehetőségek használatával.

Éppen ezért, újra lementjük a szem DataFrame-ünket, mert többször is fel fogjuk használni.

In [54]:
szem_copy = szem.copy()

### **5.1 Min-max skálázás**

 Adott attribútum $v$ értékét transzformálja lineáris módon a $[new\_min ,new\_max]$ tartományra:

$v' = \frac{v-old\_min}{old\_max-old\_min}*(new\_max-new\_min)+new\_min$


- **Végrehajtása manuálisan:**


**Blokk 5.1** Skálázzuk a *szem* adathalmaz numerikus értékeit a [0, 1] intervallumba a min-max skálázás módszerével!

In [55]:
cols = ['testsuly', 'testmagassag', 'eletkor']

for col in cols:
    col_minmax = col + '_minmaxscore'
    szem[col_minmax] = (szem[col] - szem[col].min())/(szem[col].max()-szem[col].min())
szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev,testsuly_minmaxscore,testmagassag_minmaxscore,eletkor_minmaxscore
595,596,84.000000,170.0,45.0,29.065744,fiatal,0.519780,0.492754,0.333333
124,125,80.000000,170.0,43.0,27.681661,fiatal,0.471758,0.492754,0.308642
727,728,78.000000,170.0,85.0,26.989619,idos,0.447747,0.492754,0.827160
546,547,96.000000,170.0,72.0,33.217993,kozepkoru,0.663846,0.492754,0.666667
284,285,94.000000,170.0,75.0,32.525952,idos,0.639835,0.492754,0.703704
462,463,81.000000,170.0,77.0,28.027682,idos,0.483763,0.492754,0.728395
600,601,84.000000,170.0,27.0,29.065744,fiatal,0.519780,0.492754,0.111111
251,252,68.000000,170.0,86.0,23.529412,idos,0.327692,0.492754,0.839506
477,478,55.000000,170.0,50.0,19.031142,kozepkoru,0.171620,0.492754,0.395062
719,720,40.704845,136.0,22.0,22.007377,fiatal,0.000000,0.000000,0.049383


- **Végrehajtása a preprocessing használatával:**


**Blokk 5.2** Skálázzuk a *szem* adathalmaz numerikus értékeit a [0, 1] intervallumba a min-max skálázás módszerével, de alkalmazzuk a preprocessing nyújtotta lehetőségeket!

In [56]:
from sklearn.preprocessing import MinMaxScaler

szem = szem_copy.copy()

# Az oszlopok, amelyeket skálázni szeretnénk
cols = ['testsuly', 'testmagassag', 'eletkor']

# MinMaxScaler inicializálása
scaler = MinMaxScaler()

# Az adatok átskálázása
szem[cols] = scaler.fit_transform(szem[cols])

szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,0.519780,0.492754,0.333333,29.065744,fiatal
124,125,0.471758,0.492754,0.308642,27.681661,fiatal
727,728,0.447747,0.492754,0.827160,26.989619,idos
546,547,0.663846,0.492754,0.666667,33.217993,kozepkoru
284,285,0.639835,0.492754,0.703704,32.525952,idos
462,463,0.483763,0.492754,0.728395,28.027682,idos
600,601,0.519780,0.492754,0.111111,29.065744,fiatal
251,252,0.327692,0.492754,0.839506,23.529412,idos
477,478,0.171620,0.492754,0.395062,19.031142,kozepkoru
719,720,0.000000,0.000000,0.049383,22.007377,fiatal


### **5.2 Zéruspont standardizáció**

- **Végrehajtása manuálisan:**


**Blokk 5.3** Standardizáljuk a numerikus adatokat zéruspont standardizálással!

In [57]:
szem = szem_copy.copy()

for col in cols:
    col_zscore = col + '_znorm'
    szem[col_zscore] = (szem[col] - np.mean(szem[col]))/np.std(szem[col])
szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev,testsuly_znorm,testmagassag_znorm,eletkor_znorm
595,596,84.000000,170.0,45.0,29.065744,fiatal,-0.141298,-0.026222,-0.492982
124,125,80.000000,170.0,43.0,27.681661,fiatal,-0.437340,-0.026222,-0.582047
727,728,78.000000,170.0,85.0,26.989619,idos,-0.585361,-0.026222,1.288319
546,547,96.000000,170.0,72.0,33.217993,kozepkoru,0.746828,-0.026222,0.709396
284,285,94.000000,170.0,75.0,32.525952,idos,0.598807,-0.026222,0.842994
462,463,81.000000,170.0,77.0,28.027682,idos,-0.363330,-0.026222,0.932059
600,601,84.000000,170.0,27.0,29.065744,fiatal,-0.141298,-0.026222,-1.294567
251,252,68.000000,170.0,86.0,23.529412,idos,-1.325467,-0.026222,1.332851
477,478,55.000000,170.0,50.0,19.031142,kozepkoru,-2.287604,-0.026222,-0.270319
719,720,40.704845,136.0,22.0,22.007377,fiatal,-3.345596,-3.492290,-1.517230


- **Végrehajtása a preprocessing használatával:**

**Blokk 5.4** Alkalmazzunk z-score standardizációt az összes numerikus adatunkon!

In [58]:
from sklearn.preprocessing import StandardScaler

szem = szem_copy.copy()

# Az oszlopok, amelyeket skálázni szeretnénk
cols = ['testsuly', 'testmagassag', 'eletkor']

# MinMaxScaler inicializálása
scaler = StandardScaler()

# Az adatok standardizálása
szem[cols] = scaler.fit_transform(szem[cols])

szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,-0.141298,-0.026222,-0.492982,29.065744,fiatal
124,125,-0.437340,-0.026222,-0.582047,27.681661,fiatal
727,728,-0.585361,-0.026222,1.288319,26.989619,idos
546,547,0.746828,-0.026222,0.709396,33.217993,kozepkoru
284,285,0.598807,-0.026222,0.842994,32.525952,idos
462,463,-0.363330,-0.026222,0.932059,28.027682,idos
600,601,-0.141298,-0.026222,-1.294567,29.065744,fiatal
251,252,-1.325467,-0.026222,1.332851,23.529412,idos
477,478,-2.287604,-0.026222,-0.270319,19.031142,kozepkoru
719,720,-3.345596,-3.492290,-1.517230,22.007377,fiatal


### **5.3 L-norma normalizáció**


Tipusai:

           L1- Least Absulte Deviations (L1 norma alapján):
                  - nem érzékeny az outlierekre
                  - egy adott sorban a normalizált adatok abszolút értékeinek összege 1
                  - használata: erőteljesebb vizsgálatot add, mivel nem veszi figyelembe a kiugró értékeket
                   
           L2- Least Squares (L2 norma alapján):
                  - figyelembe veszi az outlierek-et
                  - adott sorban lévó adatok négyzetösszege 1
                  - használata: adatok vizsgálata során figyelembe szeretnénk venni az outlierek-et


A módszer azonban nem tud *NaN* értékeket kezelni, ezért vagy manuálisan kell megoldanunk, vagy előtte az *NaN* értékeket megszűntetni.

**Blokk 5.5** Alkalmazzunk L1-normalizációt az adatainkon! (Előtte azonban olvassuk be újra az adatokat!)

In [59]:
from sklearn.preprocessing import normalize

szem = szem_copy.copy()

# Az oszlopok, amelyeket normalizálni szeretnénk
cols = ['testsuly', 'testmagassag', 'eletkor']

# L1 normalizáció alkalmazása az oszlopokra
szem[cols] = normalize(szem[cols], norm='l1')

szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,0.280936,0.568562,0.150502,29.065744,fiatal
124,125,0.273038,0.580205,0.146758,27.681661,fiatal
727,728,0.234234,0.510511,0.255255,26.989619,idos
546,547,0.284024,0.502959,0.213018,33.217993,kozepkoru
284,285,0.277286,0.501475,0.221239,32.525952,idos
462,463,0.246951,0.518293,0.234756,28.027682,idos
600,601,0.298932,0.604982,0.096085,29.065744,fiatal
251,252,0.209877,0.524691,0.265432,23.529412,idos
477,478,0.200000,0.618182,0.181818,19.031142,kozepkoru
719,720,0.204851,0.684432,0.110717,22.007377,fiatal


**Blokk 5.6** Alkalmazzunk L2-normalizációt az adatainkon!

In [60]:
from sklearn.preprocessing import normalize

szem = szem_copy.copy()

# Az oszlopok, amelyeket normalizálni szeretnénk
cols = ['testsuly', 'testmagassag', 'eletkor']

# L2 normalizáció alkalmazása az oszlopokra
szem[cols] = normalize(szem[cols], norm='l2')

szem.head(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
595,596,0.431019,0.872300,0.230903,29.065744,fiatal
124,125,0.415065,0.882014,0.223098,27.681661,fiatal
727,728,0.379658,0.827459,0.413729,26.989619,idos
546,547,0.461347,0.816968,0.346010,33.217993,kozepkoru
284,285,0.451417,0.816393,0.360173,32.525952,idos
462,463,0.398141,0.835605,0.378480,28.027682,idos
600,601,0.438566,0.887574,0.140968,29.065744,fiatal
251,252,0.336156,0.840390,0.425139,23.529412,idos
477,478,0.296432,0.916246,0.269484,19.031142,kozepkoru
719,720,0.283351,0.946710,0.153144,22.007377,fiatal


## **6. Mintavételezés**

**Blokk 6.1** **Visszatevés nélküli mintát** a **sample** metódussal tudunk kiválasztani. Lehetőségünk van egy meghatározott elemszámú mintát, illetve a teljes adathalmaz valahány százalékát is kiválasztani.

In [61]:
szem.sample(10)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
623,624,0.420290,0.899102,0.122363,27.660096,fiatal
182,183,0.423579,0.763426,0.487608,35.796046,idos
458,459,0.414867,0.849489,0.325967,28.393726,kozepkoru
300,301,0.404087,0.887993,0.219504,25.564954,fiatal
277,278,0.438112,0.855362,0.276428,31.231410,kozepkoru
322,323,0.431988,0.786315,0.441696,33.912513,idos
207,208,0.469135,0.818687,0.331154,32.192905,kozepkoru
216,217,0.442824,0.802922,0.399028,33.425161,idos
103,104,0.464455,0.823570,0.325597,32.787994,kozepkoru
372,373,0.379600,0.889349,0.254875,26.026175,kozepkoru


In [62]:
szem.sample(frac=0.02)

,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
233,234,0.450285,0.865934,0.217720,29.714286,fiatal
457,458,0.426737,0.791776,0.437019,34.997470,idos
643,644,0.469273,0.778897,0.416057,37.421396,idos
5,6,0.477271,0.861869,0.171447,29.772228,fiatal
554,555,0.469776,0.835660,0.284576,30.387144,kozepkoru
86,87,0.460002,0.774713,0.433840,33.929778,idos
556,557,0.396166,0.873736,0.282201,28.162494,kozepkoru
156,157,0.449879,0.880409,0.149960,28.076319,fiatal
161,162,0.468911,0.828263,0.306764,29.954369,kozepkoru
598,599,0.412435,0.879137,0.238778,28.959000,fiatal


**Blokk 6.2** **Visszatevéses mintavételezés**: az **iloc** és az **np.random.choice** függvények segítségével valósítható meg. Előtte az indexeket érdemes rendbetenni, hogy folytonosak legyenek...

In [63]:
szem.reset_index(inplace=True)
szem.iloc[np.random.choice(szem.index, 10)]

,index,id,testsuly,testmagassag,eletkor,bmi,korcsopnev
440,754,755,0.448241,0.807768,0.382873,32.075913,idos
385,218,219,0.441477,0.867730,0.228350,29.752744,fiatal
170,334,335,0.426391,0.789793,0.440927,33.121307,idos
195,27,28,0.395724,0.901371,0.175877,26.769780,fiatal
297,745,746,0.420054,0.801921,0.424827,31.179138,idos
292,304,305,0.439383,0.806340,0.395927,32.629352,idos
546,153,154,0.455011,0.875401,0.163210,29.365763,fiatal
471,193,194,0.425747,0.892531,0.148755,27.414454,fiatal
670,416,417,0.465803,0.794857,0.388882,31.506533,idos
360,743,744,0.449392,0.858389,0.247418,30.795848,kozepkoru


## **7. Adatok mentése**

**Blokk 7.1** Ha már ennyit dolgoztunk vele, akkor a tisztított migrált adathalmazt írjuk ki egy csv fájlba.

In [64]:
szem.to_csv('szemely_migralt.csv',
            sep=';', header=True, index=False)